In [63]:
import glob
from bs4 import BeautifulSoup
import requests
import string
import pandas as pd
import requests
import os

In [30]:
path = "lexile/raw/lists/"
files = [f for f in glob.glob(path + "*.html", recursive=True)]
base_url = 'https://www.commonsensemedia.org/book-reviews'

In [31]:
titles = []
urls = []
titles_edited = []

In [32]:
def get_titles(soup):
    for s in soup.findAll(class_="views-field views-field-field-reference-review-ent-prod result-title"):
        titles.append(s.get_text().strip())
    return(titles)

In [33]:
def edit_titles(titles):
    for t in titles:
        stripper = str.maketrans("", "", string.punctuation)
        titles_edited.append(t.translate(stripper).replace(" ", "-").lower())
    return(titles_edited)

In [34]:
def make_urls(title_edited):
    for t in title_edited:
        urls.append(base_url + "/" + t)
    return(urls)

In [35]:
for file in files:
    with open(file, 'r') as f: 
        f = f.read()
        soup = BeautifulSoup(f, 'html.parser')
        get_titles(soup)

In [36]:
print(len(titles))
print(titles[-10])

5816
I Promise I'll Find You


In [37]:
titles_edited = []
titles_edited = edit_titles(titles)

In [38]:
print(len(titles_edited))
print(titles_edited[-10])

5816
i-promise-ill-find-you


In [39]:
urls = make_urls(titles_edited)

In [40]:
print(len(urls))
print(urls[-10])

5816
https://www.commonsensemedia.org/book-reviews/i-promise-ill-find-you


In [58]:
df = pd.DataFrame()
df['title'] = titles
df['title_edited'] = titles_edited
df['url'] = urls
df.head(5)

,title,title_edited,url
0,Joshua Dread,joshua-dread,https://www.commonsensemedia.org/book-reviews/...
1,Abe Lincoln's Dream,abe-lincolns-dream,https://www.commonsensemedia.org/book-reviews/...
2,Teeth,teeth,https://www.commonsensemedia.org/book-reviews/...
3,Nancy Clancy: Secret Admirer,nancy-clancy-secret-admirer,https://www.commonsensemedia.org/book-reviews/...
4,On the Road to Mr. Mineo's,on-the-road-to-mr-mineos,https://www.commonsensemedia.org/book-reviews/...


# Scrape Each Book's Review Page

In [45]:
!mkdir lexile/raw/books

mkdir: cannot create directory ‘lexile/raw/books’: File exists


In [114]:
for i, j in df.iterrows():
    page = requests.get(j.url)
    soup = BeautifulSoup(page.text, 'html.parser')
    exists = os.path.isfile(path + j.title_edited + ".html")
    if exists:
        continue
    else:
        with open("lexile/raw/books/" + j.title_edited + ".html", 'wb') as f:
            try:
                f.write(soup.encode('utf-8'))
            except:
                with open("lexile/raw/book-errors.txt", 'a+') as f:
                    f.write(j.title + "\n")

KeyboardInterrupt: 